<a href="https://colab.research.google.com/github/vineet-codes/defi-exploration/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
def get_usernames(r=2):
  inputstring = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0123456789'
  # print(len(inputstring))
  username_list =list(set([inputstring[x:y] for x, y in itertools.combinations(range(len(inputstring) + 1), r = r)]))
  return username_list

len(get_usernames())

2015

Questions: 

1. How does unstake work [user has to claim it ?]

In [3]:
class Vault:

  def __init__(self, name):
    # pool is of the form {user: [(amount_1, tick_1), (amount_2, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick is used for having a contcept of time in our analysis
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def balanceOf(self, user):
    user_txns = self.pool.get(user,[])
    if(len(user_txns) == 0):
      return 0 #f'It seems {user} has made no txns in to the Vault'
    return sum([amount for (amount, _) in user_txns])
    

  def shareOfUser(self, user):
    """
    return: users shares of the total pool
    """
    return self.balanceOf(user) / self.tvl()

  def deposit(self, amount, user):
    """
    return: tuple (user, updated balance)
    """
    if (user in self.pool.keys()):
      user_txns = self.pool.get(user, []) 
      user_txns.append((amount, self.tick))
      self.pool[user] = user_txns
    else:
      self.pool[user] = [(amount, self.tick)]
    return user, self.pool[user]

  def withdraw(self, withdraw_amount, user):
    """
    return: tuple (user, updated balance)
    """
    if(user not in self.pool.keys()):
      return f'{user} has not deposited in the vault'
    
    if(self.balanceOf(user) < withdraw_amount):
      return f'Not enough balance, {user} can one withdraw upto: {self.balanceOf(user)}'
    
    # db update
    # self.pool[user] = self.pool[user] - withdraw_amount
    self.pool[user].append((-1* withdraw_amount, self.tick))

    return user,self.balanceOf(user)

  def tvl(self):
    result = 0
    for user, txns in self.pool.items():
      user_balance = self.balanceOf(user)
      result += user_balance
    return result
      
    return sum([v for _,v in self.pool.items()])
  
  def get_pool(self):
    return self.pool



class Stakes:

  def __init__(self, name="YOP_LOCKBOX"):

    # pool is of the form {user: [(amount_1, duration_1, tick_1), (amount_2, duration_2, tick_2), (amount_1, duration_1, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick helps with time travel
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def stake(self, user, amount, duration):
    """
    duratrion: in months
    """
    # assert self.tick == tick

    user_stakes = self.pool.get(user, [])
    # print(user_stakes)
    user_stakes.append((amount, duration, self.tick))
    self.pool[user] = user_stakes
    # self.rewards_pool.tick()
    return user, self.pool.get(user, [])

  def unstake(self):
    pass


  def get_pool(self):
    return self.pool

  def pool_size(self):
    return sum([amount*duration for user in self.pool.keys() for amount, duration, _ in self.pool[user]])

  def shareOfUser(self, user):
    totalPoolSize = self.pool_size()
    if(totalPoolSize == 0):
      return 0
    userPoolSize = sum([amount*duration for amount, duration, _ in self.pool.get(user, [(0,0,0)])])
    return userPoolSize / totalPoolSize



class Rewards:

  def __init__(self, vault, stakes, 
               weights=[0.2,0.8], 
               vault_stake_reward_split = [0.75, 0.25], 
               yearly_emissions = 3891931,
               yop_token_price = 1):
    
    self.tick = 0
    
    self.yearly_emissions = yearly_emissions

    self.vault = vault
    self.stakes = stakes

    self.weights = weights
    self.vault_stake_reward_split = vault_stake_reward_split
    self.yop_token_price = yop_token_price

    # list to store history state per tick
    self.reward_per_tick_history = []

  
  def update_tick(self, step):
    self.tick += step


  def calculate_rewards(self):

    result = {}
    
    total_power = 0
    for user in list(set(self.vault.pool.keys())):
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      total_power += vault_boost_multiplier * self.vault.balanceOf(user)

    # print(total_power)

    for user in list(set(self.vault.pool.keys())):
      # print("=========")
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      # print(f'{user} vault boost multiplier is: {vault_boost_multiplier}')
      # x = (vault_boost_multiplier * self.vault.balanceOf(user) / total_power) * ((self.vault_stake_reward_split[0]/4)*self.yearly_emissions) * (self.yop_token_price)
      x = ((vault_boost_multiplier * self.vault.balanceOf(user)) / total_power) * ((0.2)*self.yearly_emissions) * (self.yop_token_price)
      # print(f"{user} is getting {(x / self.vault.balanceOf(user))*100}% APY in the vault")
      result[user] = (x / self.vault.balanceOf(user))*100
    return result
 
    
  
  def tick_fn(self, step):


    # update clocks for everyone
    self.update_tick(step)
    self.vault.update_tick(step)
    self.stakes.update_tick(step)

    # TODO: calculate the rewards earned in this tick

    # TODO: unstake and calculate stake rewards to be persisted

    # calculate the current vault reward state to be persisted
    state = {}

    # print(f"===Tick: {self.tick}====")
    # print(self.calculate_rewards())
    # print("Vault pool",self.vault.pool)
    # print("Stake Pool", self.stakes.pool)

    state[f'apys_{self.tick}'] = self.calculate_rewards()
    state['vault_pool'] = self.vault.pool.copy()
    state['stake_pool'] = self.stakes.pool.copy()

    self.reward_per_tick_history.append(state)
    # print("========")

    
    
      

  


# v = Vault()
# print(v.tvl())
# print(v.balanceOf('dfg'))
# v.deposit(5000, 'e')
# print(v.tvl())

class User:

  def __init__(self, name):
    self.name = name

  def __repr__(self):
    return f'{self.name}'

  def vault_rewards(self):
    pass

  def stake_rewards(self):
    pass

In [4]:
usdc_vault = Vault('test')
yop_stakes = Stakes()
vineet = User("Vineet")
john = User("John")
anar = User("Anar")

reward_calculator = Rewards(usdc_vault, yop_stakes)

reward_calculator.tick_fn(1)

# users make deposit into vault
usdc_vault.deposit(50_000, vineet)
# yop_stakes.stake(john, 5000, 12)


reward_calculator.tick_fn(1)

usdc_vault.deposit(50_000, vineet)

reward_calculator.tick_fn(1)

usdc_vault.deposit(60_000, john)

# users stake some yop
yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)

yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)


reward_calculator.tick_fn(1)

In [5]:
usdc_vault.pool, usdc_vault.balanceOf(anar)

({John: [(60000, 3)], Vineet: [(50000, 1), (50000, 2)]}, 0)

In [6]:
usdc_vault.shareOfUser(vineet) + usdc_vault.shareOfUser(john) == 1

True

In [7]:
usdc_vault.withdraw(100000, vineet), usdc_vault.balanceOf(vineet), usdc_vault.tvl(), yop_stakes.get_pool()

((Vineet, 0), 0, 60000, {Vineet: [(5000, 12, 3), (5000, 12, 4)]})

In [8]:
def toy_sheet_setup(steps = 100):

  # create vault and stake pool
  usdc_vault = Vault('test vault')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  # create users
  vineet, john, anar = User("Vineet"), User("John"), User("Anar")

  # make some txns in vaults and stakes in some ticks
  # tick
  rewards.tick_fn(1)

  # users make deposit into vault
  usdc_vault.deposit(670_000, vineet)
  yop_stakes.stake(john, 100_000, 5)

  #tick 
  rewards.tick_fn(1)
  
  usdc_vault.deposit(2_000_000, john)

  # users stake some yop
  # yop_stakes.stake(vineet, 5000, 12)
  rewards.tick_fn(1)

  usdc_vault.deposit(5000, anar)

  yop_stakes.stake(anar, 50000, 5)
  rewards.tick_fn(1)

  # yop_stakes.stake(anar, 5000, 12)
  rewards.tick_fn(1)



  return rewards, usdc_vault, yop_stakes, [vineet, john, anar]

In [9]:
rewards, usdc_vault, yop_stakes, [vineet, john, anar] = toy_sheet_setup()

In [10]:
usdc_vault.pool, yop_stakes.pool, rewards.tick, rewards.calculate_rewards()

({Anar: [(5000, 3)], John: [(2000000, 2)], Vineet: [(670000, 1)]},
 {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
 5,
 {Anar: 14.840511248388019,
  John: 37.94960412560259,
  Vineet: 2.7838943172431097})

In [11]:
rewards.reward_per_tick_history

[{'apys_1': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_2': {Vineet: 116.1770447761194},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {Vineet: [(670000, 1)]}},
 {'apys_3': {John: 38.24237769141653, Vineet: 2.0206934584581293},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {John: [(2000000, 2)], Vineet: [(670000, 1)]}},
 {'apys_4': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}},
 {'apys_5': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}}]

In [12]:
common_users = list(set(rewards.vault.pool.keys()).intersection(set(rewards.stakes.pool.keys())))

In [13]:
common_users

[John, Anar]

## WIP: Setup Optimization simulation

In [14]:
import random

In [29]:
def setup_sim(steps=300):

  actions = ['deposit', 'withdraw', 'stake', 'unstake']

  # instantiate vaults and staking pool
  usdc_vault = Vault('USDC_GENESIS')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  # generate random users from population of all users
  usernames = get_usernames()
  user_population = [User(username) for username in usernames]

  for tick in range(1, steps+1):
    # update the tick
    rewards.tick_fn(1)
    # for each tick
    #  pick a random user and a random action he takes
    user = random.choice(user_population[:20])
    action_name = random.choice(actions)

    print(f"{user} {action_name}'s at tick {rewards.tick}")

    # execute the action
    if(action_name == 'deposit'):
      # deposit: user deposits a random amount in vault
      usdc_vault.deposit(10000, user)
    elif (action_name == 'withdraw'):
      balance = usdc_vault.balanceOf(user)
      usdc_vault.withdraw(balance/8, user)
    elif (action_name == 'stake'):
      yop_stakes.stake(user, 1000, 12)
    else:
      # unstake
      pass
  # print(rewards.calculate_rewards())
  return rewards, user_population

In [30]:
rewards, user_population = setup_sim()

opq withdraw's at tick 1
MN withdraw's at tick 2
tuvwxyz0123456 withdraw's at tick 3
HIJKL stake's at tick 4
2345 unstake's at tick 5
ghijklmnopqrstuvwxyz012345 stake's at tick 6
NOPQRSTUVWX stake's at tick 7
PQRSTUVWXYZabscdef deposit's at tick 8
JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0 deposit's at tick 9
qrstuvwxyz01 unstake's at tick 10
uvwxyz deposit's at tick 11
uvwx deposit's at tick 12
NOPQRSTUVWX stake's at tick 13
uvwxyz unstake's at tick 14
uvwx unstake's at tick 15
KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz unstake's at tick 16
DEFGHIJKLMNOPQRSTUVWXYZabscdefgh withdraw's at tick 17
DEFGHIJKLMNOPQRSTUVWXYZabscdefgh deposit's at tick 18
gh unstake's at tick 19
jklmn unstake's at tick 20
uvwx withdraw's at tick 21
opq withdraw's at tick 22
DEFGHIJKLMNOPQRSTUVWXYZabscdefgh withdraw's at tick 23
DEFGHIJKLMNOPQRSTUVWXYZabscdefgh stake's at tick 24
tuvwxyz0123456 withdraw's at tick 25
gh withdraw's at tick 26
ghijklmnopqrstuvwxyz012345 stake's at tick 27
tuvwxyz0123456 depos

In [31]:
len(rewards.vault.pool.keys()), rewards.vault.tvl()

(20, 617509.880065918)

In [32]:
rewards.tick, rewards.vault.tick, rewards.stakes.tick

(300, 300, 300)

In [33]:
rewards.calculate_rewards()

{2345: 114.67444019783497,
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: 138.01308524701773,
 HIJKL: 228.21950933648606,
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: 124.10620839955382,
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: 129.617859882848,
 MN: 178.87610189659085,
 NOPQRSTUVWX: 90.55061336464814,
 PQRSTUVWXYZabscdef: 48.65105387752588,
 TUVWXYZabscdefghijklmnopqrstuvwxyz0123: 108.40356142071674,
 VW: 89.36839802816077,
 WXYZabs: 84.6577749726081,
 defghijklmnopqrstuvwxyz012: 104.75681446203701,
 gh: 7.661952560525677,
 ghijklmnopqrstuvwxyz012345: 141.23050673239473,
 jklmn: 136.55688131040807,
 opq: 212.17729616288543,
 qrstuvwxyz01: 126.12782856649254,
 tuvwxyz0123456: 110.32367264126448,
 uvwx: 76.12046710536823,
 uvwxyz: 115.57167719511476}

In [34]:
rewards.vault.pool

{2345: [(10000, 32),
  (-1250.0, 65),
  (10000, 81),
  (-2343.75, 110),
  (10000, 140),
  (-3300.78125, 167),
  (-2888.18359375, 183),
  (-2527.16064453125, 279),
  (10000, 287)],
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: [(10000, 18),
  (-1250.0, 23),
  (-1093.75, 146),
  (10000, 155),
  (10000, 249)],
 HIJKL: [(10000, 60),
  (-1250.0, 131),
  (10000, 141),
  (10000, 173),
  (10000, 224),
  (-4843.75, 244),
  (10000, 260),
  (10000, 269)],
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: [(10000, 9),
  (10000, 215),
  (10000, 248),
  (10000, 288)],
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: [(10000, 74),
  (-1250.0, 95),
  (-1093.75, 108),
  (-957.03125, 113),
  (10000, 135)],
 MN: [(10000, 232), (10000, 258)],
 NOPQRSTUVWX: [(10000, 42),
  (-1250.0, 78),
  (-1093.75, 196),
  (-957.03125, 208),
  (10000, 233),
  (10000, 257)],
 PQRSTUVWXYZabscdef: [(10000, 8),
  (-1250.0, 55),
  (10000, 102),
  (-2343.75, 159),
  (10000, 172),
  (-3300.78125, 210),
  (10000, 218),
  (10000, 238),
  (-5388.

In [35]:
rewards.stakes.pool

{2345: [(1000, 12, 48), (1000, 12, 99), (1000, 12, 180), (1000, 12, 229)],
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: [(1000, 12, 24),
  (1000, 12, 59),
  (1000, 12, 252),
  (1000, 12, 278),
  (1000, 12, 296)],
 HIJKL: [(1000, 12, 4),
  (1000, 12, 62),
  (1000, 12, 88),
  (1000, 12, 138),
  (1000, 12, 143),
  (1000, 12, 152),
  (1000, 12, 185),
  (1000, 12, 256)],
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: [(1000, 12, 29),
  (1000, 12, 73),
  (1000, 12, 191),
  (1000, 12, 231)],
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: [(1000, 12, 41),
  (1000, 12, 220),
  (1000, 12, 237),
  (1000, 12, 282),
  (1000, 12, 284)],
 MN: [(1000, 12, 67),
  (1000, 12, 125),
  (1000, 12, 134),
  (1000, 12, 153),
  (1000, 12, 162),
  (1000, 12, 192),
  (1000, 12, 265)],
 NOPQRSTUVWX: [(1000, 12, 7), (1000, 12, 13), (1000, 12, 286)],
 PQRSTUVWXYZabscdef: [(1000, 12, 276)],
 TUVWXYZabscdefghijklmnopqrstuvwxyz0123: [(1000, 12, 226),
  (1000, 12, 241),
  (1000, 12, 291)],
 VW: [(1000, 12, 116), (1000, 12, 222), 